# Add new users in the database

- Load the data base
- Get the unique users for this year and find the on without group
- Edit the new users.csv file to add the id of the groups
- 

In [3]:
from usagestats.bookingdb import BookingDB
from datetime import datetime, timedelta
import pytz
import pandas as pd

db_file = 'bookings.db'

local_tz = pytz.timezone("Europe/London")
start_date = datetime.fromisoformat('2023-11-02').replace(tzinfo=local_tz)
end_date =  start_date + timedelta(weeks=52) # iso-format data yyyy-mm-dd
remove_bookings_type = ['service', 'maintenance']
remove_instrument = ['Analysis pc1','Analysis pc2' ,'Analysis pc3' ,'Analysis pc4','Zeiss sim analysis pc']

with BookingDB(db_file) as db:
    bookings = db.get_events_in_range(start_date, end_date)

newusers = bookings[bookings['group']=='Unknown'][['user','group','division']].drop_duplicates().reset_index()
print(len(newusers))
newusers.to_csv('new users.csv')


14


In [28]:
with BookingDB(db_file) as db:
    groups = pd.read_sql_query("SELECT id, name FROM groups",db.connection)
    
newusers = pd.read_csv('new users.csv',index_col=0)
newusers['group'] = [groups[groups['id']==x].iloc[0]['name'] for x in newusers['group']]
with BookingDB(db_file) as db:
    db.update_users_group_from_df(newusers)